In [2]:
from src.lib import state
from src.lib import ops
from src.lib import bell
from src.lib import helper

import math

In [3]:
def create_xab(alpha: float, display_state = True): 
    ab = bell.bell_state(0, 0)
    x = state.qubit(alpha, math.sqrt(1.0 - alpha * alpha))
    xab = x * ab
    if display_state: 
        print(xab)
    return xab

def alice_operations(xab: state.State) -> list: 
    '''Returns list tupels (state, probability)'''
    xab = ops.Cnot(0, 1)(xab)
    xab = (ops.Hadamard(1) * ops.Identity(2))(xab)

    # a = 0
    p_x0, x0b = ops.Measure(xab, idx=1, tostate=0, collapse=True)
        # x = 0
    temp = ops.Measure(x0b, idx=0, tostate=0, collapse=True)
    p_00, r_00b = temp[0] * p_x0 , temp[1]
        # x = 1
    temp = ops.Measure(x0b, idx=0, tostate=1, collapse=True)
    p_10, r_10b = temp[0] * p_x0, temp[1]
    
    # a = 1
    p_x1, x1b = ops.Measure(xab, idx=1, tostate=1, collapse=True)
        # x = 0
    temp = ops.Measure(x1b, idx=0, tostate=0, collapse=True)
    p_01, r_01b = temp[0] * p_x1, temp[1]
        # x = 1
    temp = ops.Measure(x1b, idx=0, tostate=1, collapse=True)
    p_11, r_11b = temp[0] * p_x1, temp[1]
    return [(r_00b, p_00), (r_01b, p_01), (r_10b, p_10), (r_11b, p_11)]

def bob_operations(xab:state.State, m_x: int = 0, m_a: int = 0, display_operation = True) -> tuple: 
    '''Returns tuple of (alpha and beta) of qubit b'''
    
    if m_x == 0 and m_a == 0:
        if display_operation: 
            print("Bob doesn't do an operation")
    if m_x == 0 and m_a == 1:
        if display_operation: 
            print("Bob aplies X")
        xab = (ops.Identity(2) * ops.PauliX())(xab)
    if m_x == 1 and m_a == 0:
        if display_operation: 
            print("Bob aplies Z")
        xab = (ops.Identity(2) * ops.PauliZ())(xab)
    if m_x == 1 and m_a == 1:
        if display_operation: 
            print("Bob applies X and Z")
        xab = (ops.Identity(2) * ops.PauliX())((ops.Identity(2) * ops.PauliZ())(xab))
        
    p_b_0, _ = ops.Measure(xab, idx=2, tostate=0, collapse=False)
    p_b_1, _ = ops.Measure(xab, idx=2, tostate=1, collapse=False)
    return round(math.sqrt(p_b_0.real), 5), round(math.sqrt(p_b_1.real), 5)

In [4]:
x_aplha = float(input('Enter value for alpha of x [2 = sqrt(2)]'))
if round(x_aplha) == 2: 
    x_aplha = math.sqrt(1/2)
xab = create_xab(x_aplha, False)
args = alice_operations(xab)
print(f'Alpha of Qubit x was: {round(x_aplha, 5)}')
for idx, val in enumerate(args): 
    curr_state = val[0]
    prob = val[1]
    bits = helper.val2bits(idx, 2)
    alpha, beta = bob_operations(curr_state, bits[0], bits[1], False)
    print(f'With a probability of {round(prob,2)} alice measures xa = {bits}. Bob creates Quibit with alpha {alpha} and beta {beta}.')

Alpha of Qubit x was: 0.70711
With a probability of 0.25 alice measures xa = [0, 0]. Bob creates Quibit with alpha 0.70711 and beta 0.70711.
With a probability of 0.25 alice measures xa = [0, 1]. Bob creates Quibit with alpha 0.70711 and beta 0.70711.
With a probability of 0.25 alice measures xa = [1, 0]. Bob creates Quibit with alpha 0.70711 and beta 0.70711.
With a probability of 0.25 alice measures xa = [1, 1]. Bob creates Quibit with alpha 0.70711 and beta 0.70711.
